In [28]:
import os
import os.path as op
import argparse
import pandas as pd
import numpy as np
from nilearn import image

bids_folder = '/Users/mrenke/data/ds-smilepilot'
bids_folder = '/Users/mrenke/data/ds-dnumrisk'

sourcedata = op.join(bids_folder, 'sourcedata')

In [29]:
subject = '01'
session = 1
run = 1
task = 'magjudge'
n_volumes = 188 #135



In [30]:
behavior = pd.read_table(op.join(sourcedata, f'behavior_magjudge/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_task-{task}_run-{run}_events.tsv'))


In [31]:
behavior

,trial_nr,onset,event_type,phase,response,nr_frames,n1,n2,choice,onset_abs,duration
0,1.0,0.075341,stim,0,NaN,621.0,NaN,NaN,NaN,4.616172,10.346550
1,1.0,10.389703,response,0,1,NaN,NaN,NaN,NaN,14.930535,NaN
2,0.0,10.421891,stim,0,NaN,1944.0,NaN,NaN,NaN,14.962722,32.430691
3,0.0,42.835761,pulse,0,5,NaN,NaN,NaN,NaN,47.376593,NaN
4,0.0,42.852582,stim,1,NaN,137.0,NaN,NaN,NaN,47.393413,2.284712
...,...,...,...,...,...,...,...,...,...,...,...
432,31.0,463.374468,pulse,0,5,NaN,NaN,NaN,NaN,467.915299,NaN
433,31.0,465.659920,pulse,0,5,NaN,NaN,NaN,NaN,470.200751,NaN
434,31.0,467.962180,pulse,0,5,NaN,NaN,NaN,NaN,472.503011,NaN
435,31.0,470.264363,pulse,0,5,NaN,NaN,NaN,NaN,474.805194,NaN


In [32]:

behavior['trial_nr'] = behavior['trial_nr'].astype(int)

behavior = behavior[behavior['trial_nr'] > 0] # remove beginning "trial" of each run
# print(behavior)
pulses = behavior[behavior.event_type == 'pulse'][['trial_nr', 'onset']]

print(pulses)



     trial_nr       onset
14          1   52.027500
15          1   54.313332
16          1   56.615093
21          1   58.916877
22          1   61.219075
..        ...         ...
432        31  463.374468
433        31  465.659920
434        31  467.962180
435        31  470.264363
436        31  472.566456

[184 rows x 2 columns]


In [33]:
pulses['ipi'] = pulses['onset'].diff() ## this is the problem .. or not?

print(pulses)


     trial_nr       onset       ipi
14          1   52.027500       NaN
15          1   54.313332  2.285832
16          1   56.615093  2.301761
21          1   58.916877  2.301784
22          1   61.219075  2.302197
..        ...         ...       ...
432        31  463.374468  2.301987
433        31  465.659920  2.285452
434        31  467.962180  2.302260
435        31  470.264363  2.302183
436        31  472.566456  2.302093

[184 rows x 3 columns]


In [34]:
pulses = pulses[((pulses['ipi'] > 1.) & (pulses['ipi'] < 5.)) | pulses.ipi.isnull()]
print(pulses.sort_values('ipi')['ipi'])

265    2.285172
180    2.285189
285    2.285209
123    2.285241
74     2.285254
         ...   
327    2.302359
264    2.302369
270    2.302403
83     2.302404
14          NaN
Name: ipi, Length: 184, dtype: float64


In [35]:
if n_volumes != pulses.shape[0]:
    pulses = pulses.set_index(np.arange(1, pulses.shape[0]+1))[['trial_nr', 'onset']]
    t0 = pulses.loc[1, 'onset'] - (n_volumes - pulses.shape[0]) * 2.3
    print(f'******Pulse missing: {pulses.loc[1, "onset"]}, {t0} ({pulses.shape[0]})*******')
else:
    pulses = pulses.set_index(np.arange(1, n_volumes+1))[['trial_nr', 'onset']]
    t0 = pulses.loc[1, 'onset']
    print(t0)



******Pulse missing: 52.02749979999953, 42.82749979999953 (184)*******


In [6]:
# phase 2 = stim 1
# phase 4 = stim 2
stim1 = behavior[(behavior['event_type'] == 'stim') & (behavior['phase'] == 2)].copy()

stim1['n'] = stim1['n1']
stim1['onset'] -= t0
stim1['trial_type'] = 'stimulus 1'


stim2 = behavior[(behavior['event_type'] == 'stim') & (behavior['phase'] == 4)].copy()
stim2['n'] = stim2['n2']
stim2['onset'] -= t0
stim2['trial_type'] = 'stimulus 2'


choice = behavior[(behavior['event_type'] == 'choice')].copy()
choice['onset'] -= t0
choice['trial_type'] = 'choice'

events = pd.concat((stim1, stim2, choice)).sort_index().reset_index(drop=True)
# result['choice'] = result['choice'].astype(int)
events = events[['trial_nr', 'onset', 'trial_type', 'n1', 'n2', 'choice']]

,trial_nr,onset,trial_type,n1,n2,choice
0,1,428.791001,stimulus 1,7.0,4.0,NaN
1,1,437.403211,stimulus 2,7.0,4.0,NaN
2,1,438.389915,choice,NaN,NaN,1.0
3,2,444.000769,stimulus 1,14.0,11.0,NaN
4,2,451.598023,stimulus 2,14.0,11.0,NaN
...,...,...,...,...,...,...
85,29,809.996176,stimulus 2,5.0,7.0,NaN
86,29,811.247045,choice,NaN,NaN,2.0
87,30,816.096612,stimulus 1,14.0,8.0,NaN
88,30,821.704913,stimulus 2,14.0,8.0,NaN
